In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_excel(r"C:\Users\HP\Downloads\archive\dataset.xlsx")
df1 = data.copy()
df1 = pd.DataFrame(df1)
missing_rate = df1.isna().sum() / df1.shape[0]
missing_rate

Patient ID                                               0.000000
Patient age quantile                                     0.000000
SARS-Cov-2 exam result                                   0.000000
Patient addmited to regular ward (1=yes, 0=no)           0.000000
Patient addmited to semi-intensive unit (1=yes, 0=no)    0.000000
                                                           ...   
HCO3 (arterial blood gas analysis)                       0.995216
pO2 (arterial blood gas analysis)                        0.995216
Arteiral Fio2                                            0.996456
Phosphor                                                 0.996456
ctO2 (arterial blood gas analysis)                       0.995216
Length: 111, dtype: float64

In [4]:
blood_columns = df1.columns[(missing_rate < 0.9) & (missing_rate > 0.8)].tolist()
viral_columns = df1.columns[(missing_rate < 0.88) & (missing_rate > 0.75)].tolist()
keys_columns = ['Patient age quantile', 'SARS-Cov-2 exam result']

In [6]:
# Vérification que toutes les colonnes existent dans df1
all_columns = set(df1.columns)
for col in keys_columns + blood_columns + viral_columns:
    if col not in all_columns:
        raise ValueError(f"La colonne '{col}' n'existe pas dans df1.")
# Concaténation et filtrage
selected_columns = keys_columns + blood_columns + viral_columns
df1 = df1[selected_columns]
df1.head()
df1.shape

(5644, 41)

In [7]:
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(df1, test_size=0.2, random_state=0)
trainset['SARS-Cov-2 exam result'].value_counts()
testset['SARS-Cov-2 exam result'].value_counts()

SARS-Cov-2 exam result
0    1018
1     111
Name: count, dtype: int64

In [64]:
def encodage(df1):
    code = {
        'negative': 0,
        'positive': 1,
        'not_detected': 0,
        'detected': 1
    }
    
    # Appliquer le remplacement uniquement aux colonnes de type object
    for col in df1.select_dtypes(include="object"):
        df1[col] = df1[col].replace(code)
        
    return df1

In [65]:
 def imputation(df1): 
    df1 = df1.dropna(axis=0)
    return df1

In [69]:
def preprocessing(df1):
    df1 = encodage(df1)
    df1 = imputation(df1)
    
    x= df1.drop('SARS-Cov-2 exam result', axis=1)
    y = df1['SARS-Cov-2 exam result']
    print(y.value_counts())
    return x, y 

In [70]:
df1.dtypes.value_counts()

float64    39
int64       2
Name: count, dtype: int64

In [72]:
x_train, y_train = preprocessing(trainset)

SARS-Cov-2 exam result
0    73
1    10
Name: count, dtype: int64


In [73]:
x_test, y_test = processing(testset) 

SARS-Cov-2 exam result
0    13
1     3
Name: count, dtype: int64


modelisation 

In [76]:
from sklearn.tree import DecisionTreeClassifier  # <-- Correction ici

model = DecisionTreeClassifier(random_state=0)

In [77]:
from sklearn.metrics import f1_score,confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import learning_curve


In [80]:
def evaluation(model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred))     

In [81]:
evaluation(model)

[[11  2]
 [ 1  2]]
              precision    recall  f1-score   support

           0       0.92      0.85      0.88        13
           1       0.50      0.67      0.57         3

    accuracy                           0.81        16
   macro avg       0.71      0.76      0.73        16
weighted avg       0.84      0.81      0.82        16

0.5714285714285714
